In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [2]:
import ee 
import geemap

ee.Initialize(project="ee-india-reservoirs")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [4]:
from src.et_blue.exporting_utils import postprocess_et_blue_raw, process_et_blue_raw
from src.et_blue.compute_et_blue import compute_et_blue, compute_volumetric_et_blue, postprocess_et_blue

from src.et_green.compute_et_green import calculate_band_std_dev

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
)

from utils.date_utils import print_collection_dates

---

## Constants

In [41]:
YEAR = 2018

PATH_TO_AOI = "projects/thurgau-irrigation/assets/FribourgAndVaud/broye_bounding_box"
# PATH_TO_ET_GREEN = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_dekadal_from_Landsat_30m_v2"
TEMPORAL_RESOLUTION = "dekadal"
# ET_PRODUCT = "Landsat_30m"
ET_PRODUCT = "WaPOR_10m"

PATH_TO_ET_GREEN = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_{TEMPORAL_RESOLUTION}_from_{ET_PRODUCT}_v2"

# PATH_TO_ET_PRODUCT = (f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_products/decadal_Landsat_10m")
PATH_TO_ET_PRODUCT = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_products/WaPOR_10m_{YEAR}"

# #Landsat
# ET_GREEN_BAND_NAME = "ET_green"
# ET_BAND_NAME = "ET"
# SCALING_FACTOR = 100
# DYNAMIC = True
# SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
# NUMBER_OF_IMAGES = 21

##Wapor
ET_BAND_NAME ="downscaled"
SCALING_FACTOR = 100  # scaling factor for the ET band to convert it back to float
DYNAMIC = False #True #False, for wapor
SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
ET_BAND_RESOLUTION = 10  # 30# in meters
NUMBER_OF_IMAGES = 21

# ET_BLUE_POSTPROCESSED_PATH = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v2"
ET_BLUE_POSTPROCESSED_PATH = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_{TEMPORAL_RESOLUTION}_from_{ET_PRODUCT}_v2"

print(ET_BLUE_POSTPROCESSED_PATH)
EXPORT_IMAGE_RESOLUTION = 30  # in meters

projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_dekadal_from_WaPOR_10m_v2


## 1. Load assets

In [6]:
aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)

In [42]:
et_green_collection = (
    ee.ImageCollection(PATH_TO_ET_GREEN)
    # .filterBounds(aoi)
    .filterDate(f"{YEAR}-04-01", f"{YEAR}-10-31")
    .map(lambda img: back_to_float(img, SCALING_FACTOR))
)

print_collection_dates(et_green_collection)

print(et_green_collection.first().getInfo())
et_green_list = et_green_collection.toList(et_green_collection.size())

Dates of images in the collection:
2018-04-01
2018-04-11
2018-04-21
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01
2018-07-11
2018-07-21
2018-08-01
2018-08-11
2018-08-21
2018-09-01
2018-09-11
2018-09-21
2018-10-01
2018-10-11
2018-10-21
{'type': 'Image', 'bands': [{'id': 'ET_green', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1963, 2387], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 327440, 0, -10, 5195720]}, {'id': 'ET_green_std', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1963, 2387], 'crs': 'EPSG:32632', 'crs_transform': [10, 0, 327440, 0, -10, 5195720]}], 'properties': {'system:time_start': 1522540800000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[6.743775905774364, 46.67791279248287], [6.74379230131995, 46.67791241686437], [7.000159954652322, 46.68268240154832], [7.000219018288402, 46.6827200237579], [7.000284417647323, 46.682752222712836], [7.000287991453084, 46.6827671085267

In [16]:
# et_blue_raw_collection = (
#     ee.ImageCollection(PATH_TO_ET_BLUE_RAW)
#     .filterBounds(aoi)
#     .filterDate(f"{YEAR}-04-01", f"{YEAR}-10-31")
#     .map(lambda img: back_to_float(img, SCALING_FACTOR))
# )

# print_collection_dates(et_blue_raw_collection)


# et_blue_raw_list = et_blue_raw_collection.toList(et_blue_raw_collection.size())

In [43]:
et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT)
    # .filter(ee.Filter.eq("Region", "Thurgau"))
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
    .map(
        lambda img: back_to_float(
            img, SCALING_FACTOR, DYNAMIC, SCALING_FACTOR_PROPERTY_NAME
        )
    )
).sort("system:time_start")

et_collection_list = et_collection.toList(et_collection.size())


# Sanity checks:
print_collection_dates(et_collection)


Dates of images in the collection:
2018-04-01
2018-04-11
2018-04-21
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01
2018-07-11
2018-07-21
2018-08-01
2018-08-11
2018-08-21
2018-09-01
2018-09-11
2018-09-21
2018-10-01
2018-10-11
2018-10-21


## 2. Computing and exporting ET blue postprocessed

In [9]:
# postprocess_et_blue_raw(
#     et_blue_raw_list=et_blue_raw_list,
#     et_green_list=et_green_list,
#     year=YEAR,
#     aoi=aoi,
#     asset_path=ET_BLUE_POSTPROCESSED_PATH,
#     time_step_type=TEMPORAL_RESOLUTION,
#     resolution=EXPORT_IMAGE_RESOLUTION,
#     et_green_band_name=ET_GREEN_BAND_NAME,
#     number_of_images=NUMBER_OF_IMAGES,
# )

In [44]:
process_et_blue_raw(
    et_collection_list=et_collection_list,
    et_green_list=et_green_list,
    year=YEAR,
    aoi=aoi,
    asset_path=ET_BLUE_POSTPROCESSED_PATH,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=EXPORT_IMAGE_RESOLUTION,
    et_green_band_name=ET_GREEN_BAND_NAME,
    number_of_images=NUMBER_OF_IMAGES,
)

Exporting ET_blue_postprocessed_dekadal_2018_04_D1 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_dekadal_from_WaPOR_10m_v2/ET_blue_postprocessed_dekadal_2018_04_D1
Exporting ET_blue_postprocessed_dekadal_2018_04_D2 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_dekadal_from_WaPOR_10m_v2/ET_blue_postprocessed_dekadal_2018_04_D2
Exporting ET_blue_postprocessed_dekadal_2018_04_D3 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_dekadal_from_WaPOR_10m_v2/ET_blue_postprocessed_dekadal_2018_04_D3
Exporting ET_blue_postprocessed_dekadal_2018_05_D1 for 2018 to projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_dekadal_from_WaPOR_10m_v2/ET_blue_postprocessed_dekadal_2018_05_D1
Exporting ET_blue_postprocessed_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/Fr

## 3. Sanity check

In [ ]:
et_blue_postprocessed = []

et_blue_previous = None
i = 1
et_green = ee.Image(et_green_list.get(i))
et_blue_present = ee.Image(et_blue_raw_list.get(i))
et = ee.Image(et_collection_list.get(i))
print(et_blue_present.get('system:index').getInfo())

# Check if the collection has "nutzung" property, otherwise rename it
# properties = landuse_collection.first().propertyNames().getInfo()


# Initialize previous for first iteration
if et_blue_previous is None:
    et_blue_previous = et_blue_present

# Calculate threshold from ET green
threshold = calculate_band_std_dev(et_green, ET_GREEN_BAND_NAME)
print(threshold.getInfo())

ET_blue_raw_dekadal_2019_04_D2
{'Uebrige Dauerwiesen (ohne Weiden)', 'Waldweiden (ohne bewaldete Flaeche)', 'Uebrige Gruenflaeche (Dauergruenflaechen), nicht beitragsberechtigt', 'Uebrige Gruenflaeche (Dauergruenflaeche), beitragsberechtigt', 'Extensiv genutzte Weiden', 'Weiden (Heimweiden, uebrige Weiden ohne Soemmerungsweiden)'}
{'Bluehstreifen fuer Bestaeuber und and. Nuetzlinge', 'uebrige offene Ackerflaeche, nicht beitragsberechtigt', 'Hecken-, Feld- und Ufergehoelze (mit Puf.str.)', 'Weide (Heimweiden, ueb. Weide ohne Soe.geb.)', 'Uebrige Flaechen innerhalb der LN, nicht beitragsberechtigt', 'Uferwiesen (ohne Weiden) entlang von Fliessg.', 'Uferwiese (o.Wei.) entlang von Fliessgew.', 'Mehrjaehrige nachwachsende Rohstoffe (Chinaschilf, usw.)', 'Andere Obstanlagen (Kiwis, Holunder usw.)', 'Rotationsbrache', 'Baumschule von Forstpflanzen ausserhalb der Forstzone', 'Hecken-, Feld- und Ufergehoelze (mit Pufferstreifen)', 'Obstanlagen (Aepfel)', 'Baumschulen von Reben', 'Uferwiese (ohn

NameError: name 'double_cropping_image' is not defined

In [9]:
et_blue_postprocessed = []

et_blue_previous = None

for i in range(NUMBER_OF_IMAGES):
    # Get current images
    et_green = ee.Image(et_green_list.get(i))
    et_blue_present = ee.Image(et_blue_raw_list.get(i))

    # Initialize previous for first iteration
    if et_blue_previous is None:
        et_blue_previous = et_blue_present

    # Calculate threshold from ET green
    threshold = calculate_band_std_dev(et_green, ET_GREEN_BAND_NAME)

    # Post process using the previous processed image
    et_blue = postprocess_et_blue(et_blue_present, et_blue_previous, threshold)

    # Compute and add volumetric band
    et_blue_m3 = compute_volumetric_et_blue(et_blue)
    et_blue = et_blue.addBands(et_blue_m3)

    # Store current processed image for next iteration
    et_blue_previous = et_blue.select("ET_blue")

    # et_blue = back_to_int(et_blue, 100)

    et_blue_postprocessed.append(et_blue)

In [10]:
image = ee.Image(et_blue_postprocessed[3])

Map = geemap.Map()

vis_params = {
    "bands": ["ET_blue"],
    "min": 0,
    "max": 1.5,
    "palette":"viridis",
}

Map.addLayer(image, vis_params, "ET_blue")

# Map.addLayer(aoi, {}, "AOI")
Map.center_object(aoi, 12)

Map

Map(center=[46.78675332740657, 6.874554339425382], controls=(WidgetControl(options=['position', 'transparent_b…

In [16]:
et_blue_postprocessed_check = ee.ImageCollection("projects/thurgau-irrigation/assets/FribourgAndVaud/ET_blue_postprocessed/ET_blue_postprocessed_2018_dekadal_from_Landsat_30m").map(lambda img: back_to_float(img, SCALING_FACTOR))

print_collection_dates(et_blue_postprocessed_check)

Dates of images in the collection:
2018-04-01
2018-04-11
2018-04-21
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01


In [17]:
et_blue_postprocessed_check_list = et_blue_postprocessed_check.toList(et_blue_postprocessed_check.size())

image_7 = ee.Image(et_blue_postprocessed_check_list.get(-2))

Map = geemap.Map()

vis_params = {
    "bands": ["ET_blue"],
    "min": 0,
    "max": 2,
    "palette": ["blue", "green", "yellow", "red"],
}

Map.addLayer(image_7, vis_params, "ET_blue")
Map.center_object(aoi, 12)
Map

Map(center=[46.78675332740657, 6.874554339425382], controls=(WidgetControl(options=['position', 'transparent_b…